In [15]:
import skfuzzy
import numpy as np

In [ ]:
# #water content [%]
# veryLow_left = 0
# veryLow_right = 15
# low_left = 16
# low_right = 29
# medium_left = 30
# medium_right = 35
# high_left = 36
# high_right = 49
# veryHigh_left = 50
# veryHigh_right = 100

# #water temperature [Celcius]
# low_left = None
# low_right = 84
# medium_left = 85
# medium_right = 93
# high_left = 94
# high_right = 99
# veryHigh = 100 

# #current [%Stall]
# low_left = 0
# low_right = 29
# medium_left = 30
# medium_right = 69
# high_left = 70
# high_right = 100

In [254]:
class FuzzyDecisionFusion():
    def __init__(self):
        self.classes = ['Liquid','Oobleck','Soft dough','Dough','Dry dough','Crumbs']
    
    def set_params(self,params):
        self.content = np.array([params[0]])
        self.temperature = np.array([params[1]])
        self.current = np.array([params[2]])
        
        self.membership_content()
        self.membership_temperature()
        self.membership_current()
        
    def membership_content(self):
        veryLow_trapL = np.array([0,0,15,20])
        low_tri = np.array([15,22.5,29])
        med_gauss_mean, med_gauss_sigma = (33.33,0.5)
        high_tri = np.array([36,42.5,49])
        veryHigh_trapR = np.array([50,70,100,100])
        
        p_veryLow = skfuzzy.trapmf(self.content,veryLow_trapL)
        p_low = skfuzzy.trimf(self.content,low_tri)
        p_med = skfuzzy.gaussmf(self.content,med_gauss_mean,med_gauss_sigma)
        p_high = skfuzzy.trimf(self.content,high_tri)
        p_veryHigh = skfuzzy.trapmf(self.content, veryHigh_trapR)
        
        self.possibility_content = np.array([p_veryLow,p_low,p_med,p_high,p_veryHigh])        
        self.content_segment = self.defuzz(self.possibility_content)
        
    def membership_temperature(self):
        low_trapL = np.array([0,0,84,90])
        med_tri = np.array([85,89,95])
        high_tri = np.array([94,96.5,99])
        veryHigh_single = 100
        
        p_low = skfuzzy.trapmf(self.temperature,low_trapL)
        p_med = skfuzzy.trimf(self.temperature,med_tri)
        p_high = skfuzzy.trimf(self.temperature,high_tri)
        p_veryHigh = np.array([1 if self.temperature==veryHigh_single else 0])
        self.possibility_temperature = np.array([p_low,p_med,p_high,p_veryHigh])
        self.temp_segment = self.defuzz(self.possibility_temperature)
    
    def membership_current(self):
        low_trapL = np.array([0,0,30,38])
        med_tri = np.array([30,49.5,77])
        high_trapR = np.array([60,80,100,100])
        
        p_low = skfuzzy.trapmf(self.current,low_trapL)
        p_med = skfuzzy.trimf(self.current,med_tri)
        p_high = skfuzzy.trapmf(self.current,high_trapR)
        
        self.possibility_current = np.array([p_low,p_med,p_high])
        self.current_segment = self.defuzz(self.possibility_current)
    
    def defuzz(self,membership):
        return np.argmax(membership)
        
    def decision(self):
        # Classification: Liquid [0]
        isLiquidContent = (self.content_segment in range(3,5))
        isLiquidTemperature = (self.temp_segment in range(2,4))
        isLiquidCurrent = (self.current_segment==0)
        if isLiquidContent and isLiquidTemperature and isLiquidCurrent:
            return self.classes[0]
            
        
        #Classification: Oobleck [1]
        isOobleckContent = (self.content_segment in range(3))
        isOobleckTemperature = (self.temp_segment in range(2))
        isOobleckCurrent = (self.current_segment==2)
        if isOobleckContent and isOobleckTemperature and isOobleckCurrent:
             return self.classes[1]
            
        # Classification: Soft Dough [2]
        isSoftDoughContent = (self.content_segment==2)
        isSoftDoughTemperature = (self.temp_segment in range(1,3))
        isSoftDoughCurrent = (self.current_segment==1)
        if isSoftDoughContent and isSoftDoughTemperature and isSoftDoughCurrent:
             return self.classes[2]
        
        #Classification: Dough [3]
        isDoughContent = (self.content_segment==2)
        isDoughTemperature = (self.temp_segment==3)
        isDoughCurrent = (self.current_segment==1)
        if isDoughContent and isDoughTemperature and isDoughCurrent:
             return self.classes[3]
                              
        # Classification: Dry Dough [4]
        isDryDoughContent = (self.content_segment==1)
        isDryDoughTemperature = (self.temp_segment==3)
        isDryDoughCurrent = (self.current_segment==1)
        if isDryDoughContent and isDryDoughTemperature and isDryDoughCurrent:
             return self.classes[4]
        
        #Classification: Crumbs [5]
        isCrumbsContent = (self.content_segment==0)
        isCrumbsTemperature = (self.temp_segment in range(2,4))
        isCrumbsCurrent = (self.current_segment==0)
        if isCrumbsContent and isCrumbsTemperature and isCrumbsCurrent:
             return self.classes[5]
        
        return "No Classification"
    

In [255]:
doughClassifier = FuzzyDecisionFusion()

In [278]:
#Test Cases:
test1 = [40,97,10] #expect: liquid
test2 = [100,95,0] #expect: liquid
test3 = [33,85,70] #expect: oobleck
test4 = [30,90,80] #expect: oobleck
test5 = [32,93,50] #expect: soft dough
test6 = [33,88,60] #expect: soft dough
test7 = [33,100,40] #expect: dough
test8 = [34,100,50] #expect: dough
test9 = [20,100,40] #expect: dry dough 
test10 = [25,100,50] #expect: dry dough
test11 = [5,97,10] #expect: crumbs
test12 = [10,100,0] #expect: crumbs

In [284]:
params = test1

doughClassifier.set_params(params)
doughClassifier.decision()

'Liquid'